# Mistral मॉडलों के साथ निर्माण

## परिचय

यह पाठ निम्नलिखित को कवर करेगा:  
- विभिन्न Mistral मॉडलों का अन्वेषण  
- प्रत्येक मॉडल के उपयोग-मामले और परिदृश्यों को समझना  
- कोड नमूने जो प्रत्येक मॉडल की अनूठी विशेषताओं को दिखाते हैं।


## द मिस्टрал मॉडल्स

इस पाठ में, हम 3 विभिन्न मिस्टрал मॉडल्स का अन्वेषण करेंगे:  
**मिस्टрал लार्ज**, **मिस्टрал स्मॉल** और **मिस्टрал नीमो**।

इनमें से प्रत्येक मॉडल Github मॉडल मार्केटप्लेस पर मुफ्त उपलब्ध हैं। इस नोटबुक में कोड चलाने के लिए ये मॉडल उपयोग किए जाएंगे। Github मॉडल्स का उपयोग करके [AI मॉडल्स के साथ प्रोटोटाइप बनाने](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) के बारे में अधिक जानकारी यहाँ है।


## Mistral Large 2 (2407)
Mistral Large 2 वर्तमान में Mistral का प्रमुख मॉडल है और इसे उद्यम उपयोग के लिए डिज़ाइन किया गया है।

यह मॉडल मूल Mistral Large का एक उन्नयन है जो प्रदान करता है
- बड़ा संदर्भ विंडो - 128k बनाम 32k
- गणित और कोडिंग कार्यों पर बेहतर प्रदर्शन - 76.9% औसत सटीकता बनाम 60.4%
- बहुभाषी प्रदर्शन में वृद्धि - भाषाओं में शामिल हैं: अंग्रेज़ी, फ्रेंच, जर्मन, स्पेनिश, इतालवी, पुर्तगाली, डच, रूसी, चीनी, जापानी, कोरियाई, अरबी, और हिंदी।

इन विशेषताओं के साथ, Mistral Large उत्कृष्ट है
- *रिट्रीवल ऑगमेंटेड जनरेशन (RAG)* - बड़े संदर्भ विंडो के कारण
- *फंक्शन कॉलिंग* - इस मॉडल में नेटिव फंक्शन कॉलिंग है जो बाहरी टूल्स और APIs के साथ एकीकरण की अनुमति देता है। ये कॉल्स समानांतर में या एक के बाद एक क्रमिक क्रम में की जा सकती हैं।
- *कोड जनरेशन* - यह मॉडल Python, Java, TypeScript और C++ जनरेशन में उत्कृष्ट है।


### मिस्ट्रल लार्ज 2 का उपयोग करते हुए RAG उदाहरण


इस उदाहरण में, हम Mistral Large 2 का उपयोग करके एक टेक्स्ट दस्तावेज़ पर RAG पैटर्न चला रहे हैं। प्रश्न कोरियाई में लिखा गया है और लेखक की कॉलेज से पहले की गतिविधियों के बारे में पूछता है।

यह Cohere Embeddings Model का उपयोग करके टेक्स्ट दस्तावेज़ और प्रश्न दोनों के एम्बेडिंग बनाता है। इस नमूने के लिए, यह faiss Python पैकेज का उपयोग एक वेक्टर स्टोर के रूप में करता है।

Mistral मॉडल को भेजे गए प्रॉम्प्ट में प्रश्न और उन पुनः प्राप्त किए गए खंडों दोनों को शामिल किया गया है जो प्रश्न के समान हैं। मॉडल फिर एक प्राकृतिक भाषा में उत्तर प्रदान करता है।


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small मॉडल परिवार के Mistral परिवार का एक और मॉडल है जो प्रीमियर/एंटरप्राइज श्रेणी के अंतर्गत आता है। जैसा कि नाम से पता चलता है, यह मॉडल एक छोटा भाषा मॉडल (SLM) है। Mistral Small का उपयोग करने के फायदे हैं: 
- Mistral LLMs जैसे Mistral Large और NeMo की तुलना में लागत में बचत - 80% कीमत में गिरावट
- कम विलंबता - Mistral के LLMs की तुलना में तेज़ प्रतिक्रिया
- लचीला - आवश्यक संसाधनों पर कम प्रतिबंधों के साथ विभिन्न वातावरणों में तैनात किया जा सकता है। 


Mistral Small के लिए उपयुक्त है: 
- सारांशण, भावना विश्लेषण और अनुवाद जैसे पाठ आधारित कार्य। 
- ऐसे अनुप्रयोग जहां बार-बार अनुरोध किए जाते हैं क्योंकि यह लागत प्रभावी है 
- समीक्षा और कोड सुझाव जैसे कम विलंबता वाले कोड कार्य 


## Mistral Small और Mistral Large की तुलना

Mistral Small और Large के बीच विलंबता में अंतर दिखाने के लिए, नीचे दिए गए सेल चलाएं।

आपको 3-5 सेकंड के बीच प्रतिक्रिया समय में अंतर दिखाई देना चाहिए। साथ ही एक ही प्रॉम्प्ट पर प्रतिक्रिया की लंबाई और शैली पर भी ध्यान दें।


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

इस पाठ में चर्चा किए गए अन्य दो मॉडलों की तुलना में, Mistral NeMo एकमात्र मुफ्त मॉडल है जिसके पास Apache2 लाइसेंस है।

इसे Mistral के पहले के ओपन सोर्स LLM, Mistral 7B का एक उन्नयन माना जाता है।

NeMo मॉडल की कुछ अन्य विशेषताएँ हैं:

- *अधिक कुशल टोकनाइज़ेशन:* यह मॉडल अधिक सामान्य रूप से उपयोग किए जाने वाले tiktoken के बजाय Tekken टोकनाइज़र का उपयोग करता है। यह अधिक भाषाओं और कोड पर बेहतर प्रदर्शन की अनुमति देता है।

- *फाइनट्यूनिंग:* बेस मॉडल फाइनट्यूनिंग के लिए उपलब्ध है। यह उन उपयोग मामलों के लिए अधिक लचीलापन प्रदान करता है जहाँ फाइनट्यूनिंग की आवश्यकता हो सकती है।

- *नेटिव फंक्शन कॉलिंग* - Mistral Large की तरह, इस मॉडल को फंक्शन कॉलिंग पर प्रशिक्षित किया गया है। यह इसे पहले ओपन सोर्स मॉडलों में से एक के रूप में अद्वितीय बनाता है।


## Mistral NeMo

इस पाठ में चर्चा किए गए अन्य दो मॉडलों की तुलना में, Mistral NeMo एकमात्र मुफ्त मॉडल है जिसके पास Apache2 लाइसेंस है।

इसे Mistral के पहले के ओपन सोर्स LLM, Mistral 7B का एक उन्नयन माना जाता है।

NeMo मॉडल की कुछ अन्य विशेषताएँ हैं:

- *अधिक कुशल टोकनाइज़ेशन:* यह मॉडल अधिक सामान्य रूप से उपयोग किए जाने वाले tiktoken के बजाय Tekken टोकनाइज़र का उपयोग करता है। यह अधिक भाषाओं और कोड पर बेहतर प्रदर्शन की अनुमति देता है।

- *फाइनट्यूनिंग:* बेस मॉडल फाइनट्यूनिंग के लिए उपलब्ध है। यह उन उपयोग मामलों के लिए अधिक लचीलापन प्रदान करता है जहाँ फाइनट्यूनिंग की आवश्यकता हो सकती है।

- *नेटिव फंक्शन कॉलिंग* - Mistral Large की तरह, इस मॉडल को फंक्शन कॉलिंग पर प्रशिक्षित किया गया है। यह इसे पहले ओपन सोर्स मॉडलों में से एक के रूप में अद्वितीय बनाता है।


### टोकनाइज़र की तुलना

इस उदाहरण में, हम देखेंगे कि Mistral NeMo टोकनाइज़ेशन को Mistral Large की तुलना में कैसे संभालता है।

दोनों उदाहरण एक ही प्रॉम्प्ट लेते हैं लेकिन आपको देखना चाहिए कि NeMo Mistral Large की तुलना में कम टोकन वापस करता है।


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## सीखना यहीं नहीं रुकता, यात्रा जारी रखें

इस पाठ को पूरा करने के बाद, हमारे [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) को देखें ताकि आप अपनी Generative AI ज्ञान को और बढ़ा सकें!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या अशुद्धियाँ हो सकती हैं। मूल दस्तावेज़ अपनी मूल भाषा में प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
